### Multiclass Classification Model for Tripadvisor Hotel Review Star Ratings 

Imported packages.

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


Loaded the dataset.

In [2]:

from datasets import load_dataset

ta_dataset = load_dataset("jniimi/tripadvisor-review-rating")['train']


c:\Users\rushej2\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracted data and labels.

In [3]:
train_data = [item['text'] for item in ta_dataset]
train_data_labels = [int(item['overall']) for item in ta_dataset]

Split data train:test 80:20.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_labels, test_size=0.2, random_state=42)

Created binary labels for One-vs-Rest classification.

In [5]:
def one_vs_all_labels(labels, target_class):
    return [1 if label == target_class else 0 for label in labels]

Trained 5 binary classifiers using pipeline of vectorizer and classifier.
Then, fit the classifier to the training data.


In [6]:
classifiers = {}
for star in range(1, 6):
    binary_y_train = one_vs_all_labels(y_train, star)
    

    clf = make_pipeline(
        TfidfVectorizer(analyzer='word',max_features=5000,lowercase=True),
        LogisticRegression(max_iter=1000)
    )
    
    clf.fit(X_train, binary_y_train)
    classifiers[star] = clf

Evaluated the classifiers against the test data.

In [7]:
for star in range(1, 6):
    print(f"\nEvaluation for star rating: {star}")
    binary_y_test = one_vs_all_labels(y_test, star)
    y_pred = classifiers[star].predict(X_test)
    print(classification_report(binary_y_test, y_pred))


Evaluation for star rating: 1
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     38693
           1       0.75      0.46      0.57      1566

    accuracy                           0.97     40259
   macro avg       0.86      0.73      0.78     40259
weighted avg       0.97      0.97      0.97     40259


Evaluation for star rating: 2
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     38112
           1       0.43      0.11      0.18      2147

    accuracy                           0.94     40259
   macro avg       0.69      0.55      0.57     40259
weighted avg       0.92      0.94      0.93     40259


Evaluation for star rating: 3
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     34579
           1       0.59      0.27      0.37      5680

    accuracy                           0.87     40259
   macro avg       0.74      0.62  

Found binary classifier (1 star, 2 star etc.) with highest probability for each review.

In [8]:
def predict_star_rating(classifiers, review_text):
    probabilities = {star: clf.predict_proba([review_text])[0][1] for star, clf in classifiers.items()}
    return max(probabilities, key=probabilities.get)

Predicted for the entire test set.

In [9]:
predicted_ratings = [predict_star_rating(classifiers, review) for review in X_test]

Calculated overall accuracy.

In [10]:
from sklearn.metrics import accuracy_score

overall_accuracy = accuracy_score(y_test, predicted_ratings)
print(f"Overall Accuracy: {overall_accuracy:.2f}")

Overall Accuracy: 0.65
